In [2]:
from PIL import Image, ImageDraw
from IPython.display import display
import face_recognition
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display
import cv2
import glob
    

def take_camera_shot(source=0):
    cap = cv2.VideoCapture(0) # video capture source camera (Here webcam of laptop) 
    ret,frame = cap.read() # return a single frame in variable `frame`
    cv2.destroyAllWindows()
    cap.release()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame

def load_faces_from_folder(path):
    image_list = []
    names_list = []
    for filename in glob.glob(path + '/*.jpg'):
        name = filename.split('\\')[-1].split('.')[0]
        im=face_recognition.load_image_file(filename)
        image_list.append(im)
        names_list.append(name)
    return image_list, names_list;

def encode_known_faces(known_faces_images):
    known_faces_encodings = []
    for img in known_faces_images:
        face_encoding = face_recognition.face_encodings(img)[0]
        known_faces_encodings.append(face_encoding)
    return known_faces_encodings

def initialize(path):
    known_faces_images, known_faces_names = load_faces_from_folder(path)
    known_faces_encodings = encode_known_faces(known_faces_images)
    print('Learned encoding for', len(known_faces_encodings), 'images.')
    
    return known_faces_encodings, known_faces_names;

def authorize(unknown_image, known_face_encodings, known_face_names, counter = 0):
    #unknown_image = face_recognition.load_image_file("3.jpg")

    # Find all the faces and face encodings in the unknown image
    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    # Convert the image to a PIL-format image so that we can draw on top of it with the Pillow library
    # See http://pillow.readthedocs.io/ for more about PIL/Pillow
    pil_image = Image.fromarray(unknown_image)
    # Create a Pillow ImageDraw Draw instance to draw with
    draw = ImageDraw.Draw(pil_image)

    # Loop through each face found in the unknown image
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


    # Remove the drawing library from memory as per the Pillow docs
    del draw

    # Display the resulting image
    display(pil_image)
    pil_image.save("predict" + str(counter) + ".jpg")


RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file C:\Users\Z6PLT\AppData\Local\Temp\pip-install-l23j6xdc\dlib\dlib\cuda\gpu_data.cpp:201. code: 999, reason: unknown error

In [ ]:
encodings, names = initialize("known_people")

In [ ]:
counter = 1
unknown_images, random = load_faces_from_folder("test")
for img in unknown_images:
    authorize(img, encodings, names, counter)
    counter = counter + 1

In [ ]:
authorize(take_camera_shot(), encodings, names)